In [2]:
import os
os.chdir( "../")

In [3]:
import psutil
num_cpus = psutil.cpu_count(logical=False)
num_cpus 


8

In [4]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])

metric_to_optimize="MAP"

df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [5]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [6]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seeds = [
    1811854, 3772146, 6407100, 6926227, 1340378, 
    68679, 3822303, 4058970, 1264160, 1100338, 
    6552126, 725363, 8362134, 9446122, 5256199, 
    4686339, 3830906, 7315366, 9428914, 5564236, 
    3420043, 9223036, 851434, 3300082, 7899660, 
    2053768, 4305274, 3709256, 8795594, 1418903, 
    7428566, 157832
]

seeds = seeds[:num_cpus]

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [31]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2166 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2150 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2190 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2194 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2165 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2145 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2171 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2198 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [18]:
import optuna
host = "192.168.16.83"
port = 3306
database_name = "optuna_mio"
username = "root"
password = "Japo2001"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
mysql_url
storage = optuna.storages.RDBStorage(url=mysql_url)

ials_name = "IALS_8FOLD"
knn_name = "ITEMKNN_8FOLD"
slim_name = "SLIMEN_8FOLD"
rp3beta_name = "RP3_8FOLD"
p3alpha_name = "P3Alpha_8FOLD"
ials_study = optuna.load_study(study_name=ials_name, storage=storage)
knn_study = optuna.load_study(study_name=knn_name, storage=storage)
slim_study = optuna.load_study(study_name=slim_name, storage=storage)
rp3beta_study = optuna.load_study(study_name=rp3beta_name, storage=storage)
p3alpha_study = optuna.load_study(study_name=p3alpha_name, storage=storage)

studies = [knn_study, slim_study, rp3beta_study, p3alpha_study, ials_study]

In [19]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

In [20]:
recs = []

for i in range(len(seeds)):
    split = []

    knn = ItemKNNCFRecommender(urm_train_validation[i], verbose=False)
    knn.fit(topK=knn_study.best_params["topK"], shrink=knn_study.best_params["shrink"])
    split.append(knn)

    slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train_validation[i], verbose=False)
    slim.fit(topK=slim_study.best_params["topK"], l1_ratio=slim_study.best_params["l1_ratio"], alpha=slim_study.best_params["alpha"], verbose=False, workers=12)
    split.append(slim)

    ials = ImplicitALSRecommender(urm_train_validation[i], verbose=False)
    ials.fit(use_gpu=True, num_threads=1, **ials_study.best_params)
    split.append(ials)

    rp3beta = RP3betaRecommender(urm_train_validation[i], verbose=False)
    rp3beta.fit(topK=rp3beta_study.best_params["topK"], alpha=rp3beta_study.best_params["alpha"], beta=rp3beta_study.best_params["beta"])
    split.append(rp3beta)

    p3alpha = P3alphaRecommender(urm_train_validation[i], verbose=False)
    p3alpha.fit(topK=p3alpha_study.best_params["topK"], alpha=p3alpha_study.best_params["alpha"])
    split.append(p3alpha)

    recs.append(split)

    

Similarity column 22222 (100.0%), 5220.31 column/sec. Elapsed time 4.26 sec
Using gpu: True
Similarity column 22222 (100.0%), 5379.13 column/sec. Elapsed time 4.13 sec
Using gpu: True
Similarity column 22222 (100.0%), 5264.69 column/sec. Elapsed time 4.22 sec
Using gpu: True
Similarity column 22222 (100.0%), 8931.08 column/sec. Elapsed time 2.49 sec
Using gpu: True
Similarity column 22222 (100.0%), 5687.21 column/sec. Elapsed time 3.91 sec
Using gpu: True
Similarity column 22222 (100.0%), 5393.30 column/sec. Elapsed time 4.12 sec
Using gpu: True
Similarity column 22222 (100.0%), 8696.76 column/sec. Elapsed time 2.56 sec
Using gpu: True
Similarity column 22222 (100.0%), 5774.64 column/sec. Elapsed time 3.85 sec
Using gpu: True


In [21]:
hybrid_name = "NEWHYBRID10_DB"
study = optuna.create_study(study_name=hybrid_name, storage=storage, direction="maximize", load_if_exists=True)

[I 2024-01-08 14:57:17,064] A new study created in RDB with name: NEWHYBRID10_DB


In [24]:
from Recommenders.Hybrid.GeneralizedLinearHybridRecommender import GeneralizedLinearHybridRecommender

hybrids = []

for i in range(len(seeds)):
    hybrid = GeneralizedLinearHybridRecommender(urm_train_validation[i], recs[i], verbose=False)
    hybrids.append(hybrid)



In [26]:
hybrids[0]

In [50]:
def run_fold(fold, alphas):
    hybrids[fold].fit(alphas)

    metric_result = evaluator_test[fold].evaluateRecommender(hybrids[fold])[0][metric_to_optimize].item()
    res = {'Fold': fold, metric_to_optimize: metric_result, 'params': alphas}
    #print(res)
    return res

def run_fold_wrapper(fold, alphas):
    return run_fold(fold, alphas)

sum_map = lambda results: sum(item[metric_to_optimize] for item in results)


In [51]:
from concurrent.futures import ProcessPoolExecutor


def objective_multi(trial):

    alphas= []
    for i in range(len(recs[0])):
        key = "alpha_" + str(i)
        value = trial.suggest_float(key, 0, 1)
        alphas.append(value)
    
    with ProcessPoolExecutor(max_workers=num_cpus) as executor:
        # Submit all tasks to the process pool and wait for them to complete
        #fold_results = list(executor.map(lambda fold: run_fold(fold, **current_fit_hyperparameters_dict), range(len(seeds))))
        fold_results = list(executor.map(run_fold_wrapper, range(len(seeds)), [alphas]*len(seeds)))
    #print(fold_results)
    average_score = sum_map(fold_results) / len(fold_results)
    return average_score

In [36]:
def objective(trial):

    alphas= []
    for i in range(len(recs[0])):
        key = "alpha_" + str(i)
        value = trial.suggest_float(key, 0, 1)
        alphas.append(value)

    

    score = 0
    for i, _ in enumerate(seeds):
        hybrid = hybrids[i]
        hybrid.fit(alphas)
        result, _ = evaluator_test[i].evaluateRecommender(hybrid)
        MAP_result = result["MAP"].item()
            
        score += MAP_result
    
    score = score /len(seeds)

    print(f"MAP: {score}")
       
    return score

In [52]:
study.optimize(objective_multi, n_trials=250)

EvaluatorHoldout: Processed 10467 (100.0%) in 19.46 sec. Users per second: 538
EvaluatorHoldout: Processed 10444 (100.0%) in 19.79 sec. Users per second: 528
EvaluatorHoldout: Processed 10493 (100.0%) in 20.41 sec. Users per second: 514
EvaluatorHoldout: Processed 10488 (100.0%) in 20.62 sec. Users per second: 509
EvaluatorHoldout: Processed 10440 (100.0%) in 20.52 sec. Users per second: 509
EvaluatorHoldout: Processed 10473 (100.0%) in 21.30 sec. Users per second: 492
EvaluatorHoldout: Processed 10472 (100.0%) in 21.35 sec. Users per second: 491
EvaluatorHoldout: Processed 10448 (100.0%) in 22.32 sec. Users per second: 468


[I 2024-01-08 15:09:12,501] Trial 13 finished with value: 0.048639663568097305 and parameters: {'alpha_0': 0.05060462674048993, 'alpha_1': 0.7118855087788526, 'alpha_2': 0.74379696889475, 'alpha_3': 0.40122499321090166, 'alpha_4': 0.5315116168705876}. Best is trial 11 with value: 0.04922789950623326.


EvaluatorHoldout: Processed 10444 (100.0%) in 23.73 sec. Users per second: 440
EvaluatorHoldout: Processed 10473 (100.0%) in 24.99 sec. Users per second: 419
EvaluatorHoldout: Processed 10493 (100.0%) in 25.39 sec. Users per second: 413
EvaluatorHoldout: Processed 10467 (100.0%) in 25.99 sec. Users per second: 403
EvaluatorHoldout: Processed 10488 (100.0%) in 26.62 sec. Users per second: 394
EvaluatorHoldout: Processed 10472 (100.0%) in 27.09 sec. Users per second: 387
EvaluatorHoldout: Processed 10440 (100.0%) in 27.04 sec. Users per second: 386
EvaluatorHoldout: Processed 10448 (100.0%) in 27.34 sec. Users per second: 382


[I 2024-01-08 15:09:40,637] Trial 14 finished with value: 0.04989344806203376 and parameters: {'alpha_0': 0.05689779913724857, 'alpha_1': 0.7920847635121536, 'alpha_2': 0.6529259050352075, 'alpha_3': 0.8484630577968295, 'alpha_4': 0.9416769066807087}. Best is trial 14 with value: 0.04989344806203376.


EvaluatorHoldout: Processed 10440 (100.0%) in 23.08 sec. Users per second: 452
EvaluatorHoldout: Processed 10488 (100.0%) in 25.69 sec. Users per second: 408
EvaluatorHoldout: Processed 10472 (100.0%) in 26.16 sec. Users per second: 400EvaluatorHoldout: Processed 10448 (100.0%) in 26.16 sec. Users per second: 399

EvaluatorHoldout: Processed 10473 (100.0%) in 26.51 sec. Users per second: 395
EvaluatorHoldout: Processed 10444 (100.0%) in 26.93 sec. Users per second: 388
EvaluatorHoldout: Processed 10467 (100.0%) in 26.86 sec. Users per second: 390
EvaluatorHoldout: Processed 10493 (100.0%) in 27.22 sec. Users per second: 385


...

[I 2024-01-08 16:22:24,495] Trial 260 finished with value: 0.050988381859140244 and parameters: {'alpha_0': 0.062095844131561974, 'alpha_1': 0.8860766940963797, 'alpha_2': 0.042692608056467164, 'alpha_3': 0.5688834137181914, 'alpha_4': 0.07814747012461182}. Best is trial 233 with value: 0.05102771376789832.


EvaluatorHoldout: Processed 10467 (100.0%) in 13.09 sec. Users per second: 800
EvaluatorHoldout: Processed 10473 (100.0%) in 13.64 sec. Users per second: 768EvaluatorHoldout: Processed 10488 (100.0%) in 13.64 sec. Users per second: 769

EvaluatorHoldout: Processed 10444 (100.0%) in 13.84 sec. Users per second: 754
EvaluatorHoldout: Processed 10440 (100.0%) in 13.90 sec. Users per second: 751
EvaluatorHoldout: Processed 10472 (100.0%) in 14.07 sec. Users per second: 744
EvaluatorHoldout: Processed 10448 (100.0%) in 14.27 sec. Users per second: 732
EvaluatorHoldout: Processed 10493 (100.0%) in 14.24 sec. Users per second: 737


[I 2024-01-08 16:22:38,999] Trial 261 finished with value: 0.05096915386933766 and parameters: {'alpha_0': 0.10993865176175459, 'alpha_1': 0.8411026033147502, 'alpha_2': 0.07981956582811754, 'alpha_3': 0.6204240952806305, 'alpha_4': 0.0012556164714510164}. Best is trial 233 with value: 0.05102771376789832.


EvaluatorHoldout: Processed 10488 (100.0%) in 13.03 sec. Users per second: 805
EvaluatorHoldout: Processed 10473 (100.0%) in 13.11 sec. Users per second: 799
EvaluatorHoldout: Processed 10444 (100.0%) in 13.59 sec. Users per second: 768
EvaluatorHoldout: Processed 10472 (100.0%) in 13.65 sec. Users per second: 767
EvaluatorHoldout: Processed 10493 (100.0%) in 13.65 sec. Users per second: 769
EvaluatorHoldout: Processed 10467 (100.0%) in 13.65 sec. Users per second: 767
EvaluatorHoldout: Processed 10440 (100.0%) in 13.86 sec. Users per second: 753
EvaluatorHoldout: Processed 10448 (100.0%) in 13.91 sec. Users per second: 751


[I 2024-01-08 16:22:53,105] Trial 262 finished with value: 0.05096145325146852 and parameters: {'alpha_0': 0.1481639547305953, 'alpha_1': 0.9034492083059913, 'alpha_2': 0.08720321645114512, 'alpha_3': 0.5891522679917783, 'alpha_4': 0.10218297731253478}. Best is trial 233 with value: 0.05102771376789832.


In [53]:
# studies = [knn_study, slim_study, rp3beta_study, p3alpha_study, ials_study]
for s in studies:
    print(s.best_params)

#{'topK': 15, 'shrink': 12.300684239254931}
#{'topK': 641, 'l1_ratio': 0.03280421966009752, 'alpha': 0.0017252874446017745}
#{'topK': 29, 'alpha': 0.33723025040409343, 'beta': 0.15542352567862933}
#{'topK': 65, 'alpha': 0.7555650628550881}
#{'num_factors': 401, 'epochs': 1204, 'confidence_scaling': 'linear', 'alpha': 7.139858098793055, 'epsilon': 0.10666578585393433, 'reg': 0.0010579931024405477}

{'topK': 15, 'shrink': 12.300684239254931}
{'topK': 641, 'l1_ratio': 0.03280421966009752, 'alpha': 0.0017252874446017745}
{'topK': 29, 'alpha': 0.33723025040409343, 'beta': 0.15542352567862933}
{'topK': 65, 'alpha': 0.7555650628550881}
{'num_factors': 401, 'epochs': 1204, 'confidence_scaling': 'linear', 'alpha': 7.139858098793055, 'epsilon': 0.10666578585393433, 'reg': 0.0010579931024405477}


In [54]:
#hybrid weights:
print(study.best_params)

#{'alpha_0': 0.09281383615143128, 'alpha_1': 0.9071148100464619, 'alpha_2': 0.10122791589734623, 'alpha_3': 0.6178919992064891, 'alpha_4': 0.027430265345796477}

{'alpha_0': 0.09281383615143128, 'alpha_1': 0.9071148100464619, 'alpha_2': 0.10122791589734623, 'alpha_3': 0.6178919992064891, 'alpha_4': 0.027430265345796477}


In [55]:
study.best_value

0.05102771376789832

In [57]:

knn = ItemKNNCFRecommender(urm_all, verbose=False)
knn.fit(topK=knn_study.best_params["topK"], shrink=knn_study.best_params["shrink"])

slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_all, verbose=True)
slim.fit(topK=slim_study.best_params["topK"], l1_ratio=slim_study.best_params["l1_ratio"], alpha=slim_study.best_params["alpha"], verbose=False, workers=12)

ials = ImplicitALSRecommender(urm_all, verbose=False)
ials.fit(use_gpu=True, num_threads=1, **ials_study.best_params)

rp3beta = RP3betaRecommender(urm_all, verbose=False)
rp3beta.fit(topK=rp3beta_study.best_params["topK"], alpha=rp3beta_study.best_params["alpha"], beta=rp3beta_study.best_params["beta"])

p3alpha = P3alphaRecommender(urm_all, verbose=False)
p3alpha.fit(topK=p3alpha_study.best_params["topK"], alpha=p3alpha_study.best_params["alpha"])


Similarity column 22222 (100.0%), 11368.99 column/sec. Elapsed time 1.95 sec
Using gpu: True


In [58]:
alphas_final = []
for key, value in study.best_params.items():
    alphas_final.append(value)

In [59]:
hybrid = GeneralizedLinearHybridRecommender(urm_all, [knn,slim,ials,rp3beta,p3alpha], verbose=False)
hybrid.fit(alphas_final)

In [61]:
target_users = pd.read_csv("Dataset/data_target_users_test.csv")
target_users.columns = ["UserID"]

In [62]:
item_popularity_encoded = np.ediff1d(urm_all.tocsc().indptr)
item_popularity_encoded = np.sort(item_popularity_encoded)

tar_users = target_users["UserID"].astype(int)
topPop_encoded = item_popularity_encoded[-10:]

submission = []

print(np.unique(df["UserID"].values))

for index, user in enumerate(tar_users):
    if (user not in df["UserID"].values):
        item_list_encoded = topPop_encoded
    else:
        item_list_encoded = hybrid.recommend(user2user_encoded[user])[:10]
    item_list = []
    for item_encoded in item_list_encoded:
        item_list.append(item_encoded2item[item_encoded])
    submission.append((user, item_list))


def write_submission(submissions):
    with open("./submission_SUPERHYBRID.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")
            
write_submission(submission)

[    1     2     3 ... 13022 13023 13024]
